## Runnable Interface 介绍与使用

为了尽可能简化创建自定义链的过程，Langchain 实现了一个 **[Runnable](https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable)** 协议。

许多 LangChain 组件都实现了 Runnable 协议，包括 chat models, LLMs, output parsers, retrievers, prompt templates等等。此外，还有几个用于处理可运行对象的[有用原语](https://python.langchain.com/v0.1/docs/expression_language/primitives/)。

Runnable 是一个标准接口，包括：

- stream：流式返回生成内容（chunk）
- invoke：对输入调用该链
- batch：对输入列表调用该链

不同组件的输入和输出类型有所差异:

| 组件    | 输入类型                                           | 输出类型           |
| ------------ | ----------------------------------------------------- | --------------------- |
| Prompt       | Dictionary                                            | PromptValue           |
| ChatModel    | Single string, list of chat messages or a PromptValue | ChatMessage           |
| LLM          | Single string, list of chat messages or a PromptValue | String                |
| OutputParser | The output of an LLM or ChatModel                     | Depends on the parser |
| Retriever    | Single string                                         | List of Documents     |
| Tool         | Single string or dictionary, depending on the tool    | Depends on the tool   |


所有 Runnable 对象都显式描述输入和输出 Schema，以检查输入和输出格式：

- input_schema：从Runnable的结构自动生成的输入Pydantic模型
- output_schema：从Runnable的结构自动生成的输出Pydantic模型 


### Input Schema

为了演示如何使用，下面我们创建一个超级简单的PromptTemplate + ChatModel链。

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import os

print(os.getenv('OPENAI_API_KEY'))
model = ChatOpenAI(model="gpt-4o-mini")
prompt = ChatPromptTemplate.from_template("讲个关于 {topic} 的笑话吧")
chain = prompt | model

sk-svcacct-dacL7jQKDhT1TQJzqJSVi6xOfjo4H7L6EBvqp-k_Yf0uTGVbYbfmfGNcBXrWOCUT3BlbkFJtXCcoyrPHqoz7fZH5EOZADxnDXxWsPB_aCkAr6FnJBwB0ZOcjmVG-ZD-ooZqCAA


#### schema 方法

一个描述 Runnable 接受的输入的说明。这是根据任何 Runnable 结构动态生成的 Pydantic 模型。您可以调用 .schema() 来获取 `JSONSchema` 表示。

In [2]:
# 查看 Chain 的输入类型
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [3]:
# 查看 Prompt 的输入类型（Chain的输入从 Prompt 开始，因此输入类型一致）
prompt.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [4]:
# 查看 Model 的输入类型
model.input_schema.schema()

{'title': 'ChatOpenAIInput',
 'anyOf': [{'type': 'string'},
  {'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'},
  {'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'},
     {'$ref': '#/definitions/HumanMessage'},
     {'$ref': '#/definitions/ChatMessage'},
     {'$ref': '#/definitions/SystemMessage'},
     {'$ref': '#/definitions/FunctionMessage'},
     {'$ref': '#/definitions/ToolMessage'}]}}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'ToolCall': {'title': 'ToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'object'},
    'id': {

### Output Schema

输出类型仍然可以调用 .schema() 来获取其 `JSONSchema` 表示。

In [5]:
# 查看 Chain 的输出类型
chain.output_schema.schema()

{'title': 'ChatOpenAIOutput',
 'anyOf': [{'$ref': '#/definitions/AIMessage'},
  {'$ref': '#/definitions/HumanMessage'},
  {'$ref': '#/definitions/ChatMessage'},
  {'$ref': '#/definitions/SystemMessage'},
  {'$ref': '#/definitions/FunctionMessage'},
  {'$ref': '#/definitions/ToolMessage'}],
 'definitions': {'ToolCall': {'title': 'ToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'object'},
    'id': {'title': 'Id', 'type': 'string'}},
   'required': ['name', 'args', 'id']},
  'InvalidToolCall': {'title': 'InvalidToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'string'},
    'id': {'title': 'Id', 'type': 'string'},
    'error': {'title': 'Error', 'type': 'string'}},
   'required': ['name', 'args', 'id', 'error']},
  'AIMessage': {'title': 'AIMessage',
   'description': 'Message from an AI.',
   'type': 'object',
   'proper

### Stream

使用 .stream() 方法查看（同步）流式输出结果

In [6]:
for s in chain.stream({"topic": "程序员"}):
    print(s.content, end="", flush=True)

当然可以！这里有一个关于程序员的笑话：

有一天，一位程序员走进一家酒吧，点了一杯啤酒。喝了一口后，他发现啤酒里有个苍蝇。他立刻叫来酒保，怒气冲冲地说：“你们的啤酒里怎么会有苍蝇？”

酒保淡定地回答：“没关系，程序员。你只需要把苍蝇放到调试模式，就能找到问题所在了！”

希望这个笑话能让你开心！

### Invoke
使用 .invoke() 方法单次（同步）调用

In [7]:
chain.invoke({"topic": "程序员"})

AIMessage(content='当然可以！这是一个关于程序员的笑话：\n\n有一天，程序员去参加一个聚会。朋友们都在聊天，只有他在一旁默默地看着手机。大家好奇地问他：“你在干嘛呢？”\n\n程序员抬起头，笑着说：“我在调试我的社交程序。”\n\n朋友们一脸疑惑：“社交程序？是什么？”\n\n程序员无奈地说：“就是我试图从你们的对话中获取一些‘人类交互’的算法！” \n\n大家哈哈大笑，程序员也终于加入了他们的聊天！', response_metadata={'token_usage': {'completion_tokens': 132, 'prompt_tokens': 18, 'total_tokens': 150, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'stop', 'logprobs': None}, id='run-16b171c2-3f7f-47ff-a6af-826b06237ee6-0')

### Batch
使用 .batch() 方法（同步）批量调用

In [8]:
chain.batch([{"topic": "程序员"}, {"topic": "产品经理"}, {"topic": "测试经理"}])

[AIMessage(content='当然可以！这是一个关于程序员的笑话：\n\n有一天，一个程序员走进一家酒吧，点了一杯啤酒。喝了一口后，他发现啤酒太泡了，于是对酒保说：“这杯酒太泡了，能不能给我换一杯？”\n\n酒保很无奈地回答：“你是程序员吗？泡沫是你们的特权啊！”\n\n程序员一脸茫然：“为什么？”\n\n酒保笑着说：“因为你们总是把代码写得那么复杂，泡沫自然多！”\n\n希望你喜欢这个笑话！', response_metadata={'token_usage': {'completion_tokens': 130, 'prompt_tokens': 18, 'total_tokens': 148, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'stop', 'logprobs': None}, id='run-065ced2e-3bc2-4af8-bc1d-e726e5721167-0'),
 AIMessage(content='当然可以！这是一个关于产品经理的笑话：\n\n有一天，产品经理、开发人员和设计师一起去参加一个会议。路上，他们发现了一盏神灯，擦了擦灯出来了一个精灵。\n\n精灵说：“我可以给你们每人一个愿望！”\n\n产品经理想了想说：“我希望我们的产品能更快上市！”\n\n精灵点了点头，满足了他的愿望。\n\n开发人员说：“我希望我们能有更多的时间来优化代码！”\n\n精灵又点了点头，满足了他的愿望。\n\n最后，设计师想了想，朝着精灵说：“我希望能有一个完美的设计！”\n\n精灵微微一笑，回答：“你真的想要吗？那你得再给我一个愿望！”\n\n哈哈，看来在产品经理的世界里，总是要考虑时间和资源的平衡啊！', r

In [9]:
messages = chain.batch([{"topic": "程序员"}, {"topic": "产品经理"}, {"topic": "测试经理"}])

In [10]:
# 使用 StrOutputParser 来处理 Batch 批量输出
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

for idx, m in enumerate(messages):
    print(f"笑话{idx}:\n")
    print(output_parser.invoke(m))
    print("\n")

笑话0:

当然可以！这是一个关于程序员的笑话：

有一天，程序员和他的朋友一起去钓鱼。朋友问：“你觉得钓鱼和编程有什么相似之处？”

程序员回答：“钓鱼的时候，你需要耐心等待，偶尔还要调试你的钓具；而编程的时候，你也需要耐心调试代码，直到它运行。”

朋友点点头，然后问：“那钓到鱼的时候呢？”

程序员笑了：“哦，那就和写代码一样，成功的那一刻总是让人兴奋，但之后要处理的就是‘鱼’在冰箱里能放多久的问题了！”

希望你喜欢这个笑话！


笑话1:

当然可以！这是一个关于产品经理的笑话：

有一天，产品经理、开发者和设计师一起去野营。他们决定一起搭帐篷。

产品经理说：“我们先来制定一个计划，确保一切都能按时完成！”

开发者说：“等一下，我需要先写代码来实现这个帐篷。”

设计师则说：“我觉得这个帐篷的颜色不太合适，我们需要重新设计一下。”

最后，他们搭好了帐篷，但发现里面没有空间，根本放不下任何东西。

产品经理叹了口气：“看来我们需要一个新的需求：帐篷要能容纳我们所有的梦想！”

开发者和设计师对视一眼，齐声说：“那我们需要重新评估一下预算！”

哈哈，希望这个笑话能让你开心！


笑话2:

当然可以！这是一个关于测试经理的笑话：

有一天，测试经理走进办公室，看到开发团队的成员们在讨论新功能的设计。他好奇地问：“你们在讨论什么？”

开发人员回答：“我们在设计一个可以自动测试的系统！”

测试经理笑了笑，说：“那太好了！你们能不能把这个系统设计成能够自动发现开发人员的错误？”

开发人员一愣，回答：“这个功能我们还没考虑过，不过我们可以试试。”

测试经理摇了摇头，笑着说：“不用了，自动发现错误的功能已经在你们身上实现了！”

希望这个笑话能让你开心！




## 异步操作

这些方法也有相应的异步方法，应与 `asyncio` 的 `await` 语法一起使用以进行并发操作：

- astream：异步地流式返回生成内容（chunk）
- ainvoke：异步地对输入调用该链
- abatch：异步地对输入列表调用该链
- astream_log: 在发生时会返回中间步骤，并且最终返回结果之外。
- astream_events: beta 流式传输事件，在 langchain-core 0.1.14 中引入


### Async Stream

In [11]:
async for s in chain.astream({"topic": "程序员"}):
    print(s.content, end="", flush=True)

当然可以！这是一个关于程序员的笑话：

有一天，一个程序员走进一家酒吧，点了一杯啤酒。 bartender问他：“你喝酒吗？”

程序员回答：“我不知道，我只喝代码。”

bartender好奇地问：“代码是什么味道的？”

程序员笑着说：“就像你喝醉后写的那种，还得debug！” 

希望你喜欢这个笑话！

### Async Invoke

In [12]:
await chain.ainvoke({"topic": "程序员"})

AIMessage(content='当然可以！这是一个关于程序员的笑话：\n\n有一天，一个程序员走进酒吧，点了一杯啤酒。喝了一口后，他发现啤酒里有一只苍蝇。他非常生气，跑去找酒吧老板。\n\n程序员对老板说：“你知道吗？我发现你们的啤酒里有一只苍蝇！”\n\n老板看了看，淡淡地回答：“哦，那是免费的调料。”\n\n程序员无奈地摇了摇头：“我想要的是一个‘bug fix’，不是新功能！”\n\n希望你喜欢这个笑话！', response_metadata={'token_usage': {'completion_tokens': 130, 'prompt_tokens': 18, 'total_tokens': 148, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_3de1288069', 'finish_reason': 'stop', 'logprobs': None}, id='run-39efd0aa-3ac1-46b3-a709-d1ac574ef187-0')

### Async Batch

In [13]:
await chain.abatch([{"topic": "程序员"}, {"topic": "产品经理"}, {"topic": "测试经理"}])

[AIMessage(content='当然可以！这是一个关于程序员的笑话：\n\n有一天，一个程序员走进酒吧，点了一杯啤酒。 bartender问他：“你想要什么口味的？”\n\n程序员回答：“给我一个‘Bug-free’的啤酒！”\n\nbartender困惑地说：“我们只提供正常的啤酒。”\n\n程序员摇摇头说：“那我还是要一个‘Debug’的版本吧！”\n\n希望你喜欢这个笑话！', response_metadata={'token_usage': {'completion_tokens': 99, 'prompt_tokens': 18, 'total_tokens': 117, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'stop', 'logprobs': None}, id='run-1d60b0e4-6a85-4a2a-9e62-040e5bef6944-0'),
 AIMessage(content='当然可以！这是一个关于产品经理的笑话：\n\n有一天，一个产品经理走进咖啡店，看到菜单上写着“今天的特饮：咖啡+牛奶”。他想了想，心里盘算着：“这太简单了，我应该加点功能！”\n\n于是，他对服务员说：“给我来一杯咖啡，加牛奶、糖、香草味、榛子味、再加点奶泡，最后再撒点巧克力粉！”\n\n服务员愣了一下，问：“您真的需要这么多功能吗？”\n\n产品经理微笑着说：“当然！用户总是喜欢更多的选择！”\n\n服务员无奈地摇摇头：“好的，不过你确定你要喝的不是一杯‘复杂’吗？”\n\n希望这个笑话能让你轻松一笑！', response_metadata={'token_usage': {'completion_tokens'